In [9]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine

In [10]:
# read in csv file and store in dataframe
csv_file_1 = "police-violence-racial-equity/crime_data/arrests_national_adults.csv"
adult_arrests_df = pd.read_csv(csv_file_1)
adult_arrests_df

,id,state_abbr,year,offense_code,offense_name,agencies,population,total_male,total_female,m_18,...,f_50_54,f_55_59,f_60_64,f_65p,race_agencies,race_population,white,black,asian_pacific_islander,american_indian
0,1009,NaN,2016,ASR_ARSON,Arson,13310,264534532,4509,1426,161,...,102,74.0,38.0,28.0,12581,263887632,4263,1373,103,183
1,1010,NaN,2016,ASR_AST,Aggravated Assault,13310,264534532,224176,67016,5780,...,3996,2343.0,961.0,713.0,12581,263887632,183478,94982,5365,6129
2,1011,NaN,2016,ASR_AST_SMP,Simple Assault,13310,264534532,570193,213178,14018,...,12005,6697.0,3166.0,2344.0,12581,263887632,514297,237138,12418,14376
3,1012,NaN,2016,ASR_BRG,Burglary,13310,264534532,116213,28754,7077,...,1279,621.0,286.0,170.0,12581,263887632,101778,39235,2035,1323
4,1013,NaN,2016,ASR_DIS,Disorderly Conduct,13310,264534532,180722,68577,6997,...,4085,2464.0,1098.0,928.0,12581,263887632,161655,73552,2556,9460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,416,NaN,1994,ASR_STP,Stolen Property,10713,208091172,84889,13994,8768,...,148,70.0,24.0,41.0,10282,207684879,56631,40612,875,568
640,417,NaN,1994,ASR_SUS,Suspicion,10713,208091172,8230,1465,356,...,15,9.0,5.0,8.0,10282,207684879,4508,5116,26,41
641,418,NaN,1994,ASR_VAG,Vagrancy,10713,208091172,14039,3782,922,...,37,16.0,12.0,16.0,10282,207684879,9716,7678,60,361
642,419,NaN,1994,ASR_VAN,Vandalism,10713,208091172,117143,20803,12776,...,281,128.0,78.0,112.0,10282,207684879,96043,38502,1417,1708


In [11]:
# clean dataframe and eliminate unwanted columns
adult_arrests_df = adult_arrests_df[["id", "offense_name", "population", "total_male", "total_female", "race_population", "white", "black", "asian_pacific_islander", "american_indian"]]
adult_arrests_df

,id,offense_name,population,total_male,total_female,race_population,white,black,asian_pacific_islander,american_indian
0,1009,Arson,264534532,4509,1426,263887632,4263,1373,103,183
1,1010,Aggravated Assault,264534532,224176,67016,263887632,183478,94982,5365,6129
2,1011,Simple Assault,264534532,570193,213178,263887632,514297,237138,12418,14376
3,1012,Burglary,264534532,116213,28754,263887632,101778,39235,2035,1323
4,1013,Disorderly Conduct,264534532,180722,68577,263887632,161655,73552,2556,9460
...,...,...,...,...,...,...,...,...,...,...
639,416,Stolen Property,208091172,84889,13994,207684879,56631,40612,875,568
640,417,Suspicion,208091172,8230,1465,207684879,4508,5116,26,41
641,418,Vagrancy,208091172,14039,3782,207684879,9716,7678,60,361
642,419,Vandalism,208091172,117143,20803,207684879,96043,38502,1417,1708


In [16]:
# create connection to local database
rds_connection_string = "<username>:<password>@localhost:5432/police-violence-racial-equity"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [17]:
# check for tables
engine.table_names()

['adults_arrested']

In [18]:
# use pandas to load csv converted df into database
adult_arrests_df.to_sql(name = "adults_arrested", con = engine, if_exists = "append", index = False)

In [19]:
# confirm data has been added to database by querying the adults_arrested table
pd.read_sql_query("select * from adults_arrested", con = engine)

,id,offense_name,population,total_male,total_female,race_population,white,black,asian_pacific_islander,american_indian
0,1009,Arson,264534532,4509,1426,263887632,4263,1373,103,183
1,1010,Aggravated Assault,264534532,224176,67016,263887632,183478,94982,5365,6129
2,1011,Simple Assault,264534532,570193,213178,263887632,514297,237138,12418,14376
3,1012,Burglary,264534532,116213,28754,263887632,101778,39235,2035,1323
4,1013,Disorderly Conduct,264534532,180722,68577,263887632,161655,73552,2556,9460
...,...,...,...,...,...,...,...,...,...,...
639,416,Stolen Property,208091172,84889,13994,207684879,56631,40612,875,568
640,417,Suspicion,208091172,8230,1465,207684879,4508,5116,26,41
641,418,Vagrancy,208091172,14039,3782,207684879,9716,7678,60,361
642,419,Vandalism,208091172,117143,20803,207684879,96043,38502,1417,1708
